# Machine Translation using Transformer

Небольшой пет-проект, представляющий собой модель машинного перевода с русского языка на английский с использованием архитектуры Transformer.

Этот ноутбук посвящен обучению модели. Обязательно сначала посмотрите первый ноутбук — `1-data-analysis.ipynb`

## 1. Data

Загрузим предобработанные данные

In [1]:
from datasets import load_from_disk
import torch
import seaborn as sns

sns.set_style("darkgrid")
sns.set_context("notebook")
sns.set_palette(palette='Set2')

RANDOM_SEED = 42

In [2]:
dataset = load_from_disk(
    "DATA/preprocessed-data"
)
dataset

DatasetDict({
    train: Dataset({
        features: ['src', 'trg'],
        num_rows: 4334538
    })
    valid: Dataset({
        features: ['src', 'trg'],
        num_rows: 240889
    })
    test: Dataset({
        features: ['src', 'trg'],
        num_rows: 240832
    })
})

## 2. Tokenizers

Достанем токенизаторы

In [3]:
from util import get_tokenizers

tokenizer_src, tokenizer_trg = get_tokenizers()

print(f"Len of src tokenizer: {len(tokenizer_src.vocab)}")
print(f"Len of trg tokenizer: {len(tokenizer_trg.vocab)}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Len of src tokenizer: 36002
Len of trg tokenizer: 30524


## 3. Model

Итак, пару слов об архитектуре трансформер!

Источники, опираясь на которые я реализовал эту архитектуру:
- Основная статья — [Attention is All You Need](https://arxiv.org/pdf/1706.03762)
- Статья о том, в какой последовательности применять `Layer Norm`, `Attention` и `FeedForward` — [On Layer Normalization in the Transformer Architecture](https://arxiv.org/pdf/2002.04745)
- Гитхаб репозиторий, к которому я обращался, когда что-то улетало — [bentrevett](https://github.com/bentrevett/pytorch-seq2seq/tree/main)
  
Полностью расписывать то, как работает модель, я не стану, легче и понятнее будет прочитать статью. Далее можно почитать документацию к модулям, там я тоже довольно подробно объяснял, как что работает

In [4]:
from model import Transformer

In [5]:
d_model = 256
num_heads = 8
d_k = d_model
d_v = d_model
n_layers_encoder = 6
n_layers_decoder = 6
src_vocab_size = len(tokenizer_src.vocab)
trg_vocab_size = len(tokenizer_trg.vocab)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pad_ind_src = tokenizer_src.pad_token_id
pad_ind_trg = tokenizer_trg.pad_token_id
bos_ind = tokenizer_trg.bos_token_id
eos_ind = tokenizer_trg.eos_token_id
d_ff = 4 * d_model
p_dropout = 0.1
max_length = 50
batch_size = 64


print(f"Device: {device}")

model = Transformer(
    d_model=d_model,
    num_heads=num_heads,
    d_k=d_k,
    d_v=d_v,
    n_layers_encoder=n_layers_encoder,
    n_layers_decoder=n_layers_decoder,
    src_vocab_size=src_vocab_size,
    trg_vocab_size=trg_vocab_size,
    device=device,
    pad_ind_src=pad_ind_src,
    pad_ind_trg=pad_ind_trg,
    bos_ind=bos_ind,
    eos_ind=eos_ind,
    d_ff=d_ff,
    p_dropout=p_dropout,
    max_length=max_length,
)

print(
    f"Num params: {sum(param.numel() for param in model.parameters() if param.requires_grad)}"
)

Device: cuda
Num params: 35942204


Нужно иницилизировать веса модели. Наиболее часто в трансформерах применяется [инициализация Ксавье](https://paperswithcode.com/method/xavier-initialization#:~:text=Xavier%20Initialization%2C%20or%20Glorot%20Initialization,a%20n%20o%20u%20t%20%5D) 

In [6]:
from util import xavier_init, set_seed

set_seed(RANDOM_SEED)
model.apply(xavier_init)

Transformer(
  (encoder): Encoder(
    (embedding): Embedding(36002, 256)
    (positional_encoding): Embedding(50, 256)
    (layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (layer_norm_attention): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mha): MultiHeadAttention(
          (fc_q): Linear(in_features=256, out_features=256, bias=False)
          (fc_k): Linear(in_features=256, out_features=256, bias=False)
          (fc_v): Linear(in_features=256, out_features=256, bias=False)
          (fc_o): Linear(in_features=256, out_features=256, bias=False)
          (softmax): Softmax(dim=3)
        )
        (layer_norm_ff): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff): FeedForward(
          (fc_1): Linear(in_features=256, out_features=1024, bias=True)
          (relu): ReLU()
          (fc_2): Linear(in_features=1024, out_features=256, bias=True)
        )
        (dropout_attention): Dropout(p=0.1, inplace=False)
        (dropout_ff): Dr

## 4. DataLoaders

Для того, чтобы передать данные в модель, нам нужен специальный объект — DataLoader. Он соберет мини-батч из нескольких элементов (в нашем случае из 64), в котором с помощью токенизаторов создаст тензоры для `src` и `trg`, а также маски для них — `src_mask` и `trg_mask`. Логику создания DataLoader для всех трех датасетов (`train`, `valid` и `test`) я вынес в `util`.

In [7]:
from util import get_dataloaders_lazy

train_loader, valid_loader, test_loader = get_dataloaders_lazy(
    dataset, batch_size, tokenizer_src, tokenizer_trg
)

## 5. Translator class

Специальный класс, который мы будем использовать для удобства — `Translator`. Он принимает в себя нашу модель и токенизаторы и позволяет с помощью удобного интерфейса переводить строку с одного языка на другой

In [8]:
from util import Translator

translator = Translator(model, tokenizer_src, tokenizer_trg)

## 6. Training

Итак, пришло время обучить модель! Для этого я написал функцию `train`, в которой происходит много всего:
1. Если уже есть сохраненная модель, то мы достанем ее параметры
2. Проведем тренировочный цикл по одной эпохе
3. Посчитаем метрики модели на валидационной выборке
4. Сделаем шаг планировщика (`scheduler`)
5. Визуализируем loss и метрику (в данном случае практически одно и то же, т.к. используем cross entropy и perplexity)
6. Приведем примеры перевода
7. Сохраним состояние модели + сохраним ее дополнительно как лучшую, если она получила лучшее значение метрики на валидационной выборке

Обучаться будем на Cross Entropy Loss, предсказывая следующий токен:
- Encoder получает на вход всю исходную последовательность (source), которую мы маскируем для того, чтобы модель не считала attention по padding токенам
- Decoder получает целевую последовательность (target) и предсказывает для каждого токена следующий токен, основываясь на всей исходной последовательности (из Encoder) и на предшествующих токенах target'а (этого добиваемся с помощью маскирования)
- В конце модель выдает распределение вероятностей (логиты) на каждый токен для каждого токена в каждой последовательности

Значения гиперпараметров (dropout, learning rate, lr scheduler и label smoothing) я выбирал, основываясь на статьях, которые упомянул выше

In [9]:
from train import train

In [10]:
LEARNING_RATE = 5e-4
STEP_SIZE = 5
N_EPOCHS = 20
LABEL_SMOOTHING = 0.1


optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=0.1)
criterion = torch.nn.CrossEntropyLoss(ignore_index=pad_ind_trg, label_smoothing=LABEL_SMOOTHING)

train(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=criterion,
    num_epochs=N_EPOCHS,
    train_loader=train_loader,
    val_loader=valid_loader,
    clip_grad=1,
    path_to_save="DATA/model-data/training.pt",
    path_to_save_best="DATA/model-data/best.pt",
    loss_label="Cross Entropy",
    metric_label="Perplexity",
    translator=translator,
    examples_to_translate=["кошки и собаки довольно крутые"]
)

Training, epoch 1/20:   0%|          | 0/67728 [00:00<?, ?it/s]

KeyboardInterrupt: 